In [1]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
print('Done')

Done


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
train = pd.read_csv('corpus/done_twi.csv')
train.head()

,text,label,clear_text,bigram_text
0,"@first_timee хоть я и школота, но поверь, у на...",positive,"['школоть', 'поверять', 'самый', 'общество', '...","['школоть', 'поверять', 'самый', 'общество', '..."
1,"Да, все-таки он немного похож на него. Но мой ...",positive,"['да', 'таки', 'немного', 'похожий', 'но', 'ма...","['да', 'таки', 'немного', 'похожий', 'но', 'ма..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,"['ну', 'идиотка', 'испугаться']","['ну', 'идиотка', 'испугаться']"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,"['кто', 'угол', 'сидеть', 'погибать', 'голод',...","['кто', 'угол', 'сидеть', 'погибать', 'голод',..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,"['вот', 'значит', 'страшилка', 'но', 'блин', '...","['вот_значит', 'страшилка', 'но', 'блин', 'пос..."


In [3]:
len(train)

226506

In [4]:
train['label'].unique()

array(['positive', 'negative'], dtype=object)

In [5]:
train.groupby('label').nunique()

,text,clear_text,bigram_text
label,,,
negative,106859,102871,102871
positive,110259,106735,106735


In [6]:
arr = []
for item in train.bigram_text:
    temp = str(item).replace('\'', '').replace(']', '').replace('[', '')
    temp1 = temp.split(', ')
    arr.append(' '.join(temp1))


In [7]:
train['selected_text'] = arr
train = train[['selected_text','label']]
train.head()

,selected_text,label
0,школоть поверять самый общество профилировать ...,positive
1,да таки немного похожий но мальчик равный,positive
2,ну идиотка испугаться,positive
3,кто угол сидеть погибать голод еще порция взят...,positive
4,вот_значит страшилка но блин посмотреть часть ...,positive


In [8]:
data = train['selected_text']
data = np.array(data)

In [9]:
labels = np.array(train['label'])
y = []
for i in range(len(labels)):
    if labels[i] == 'neutral':
        y.append(0)
    if labels[i] == 'negative':
        y.append(1)
    if labels[i] == 'positive':
        y.append(2)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")
del y

In [10]:
len(labels)

226506

In [11]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)


[[   0    0    0 ... 1332 1112  348]
 [   0    0    0 ...   56  407  183]
 [   0    0    0 ...   31 3154 2071]
 ...
 [   0    0    0 ...  428    1   42]
 [   0    0    0 ...    0 1008  731]
 [   0    0    0 ... 2364 1486 3506]]


In [12]:
print(labels)


[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [13]:
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))


169879 56627 169879 56627


---
Single LSTM layer model
---
---

In [14]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20))
model1.add(layers.LSTM(15,dropout=0.5))
model1.add(layers.Dense(3,activation='softmax'))


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/70
5309/5309 [==============================] - 260s 49ms/step - loss: 0.6417 - accuracy: 0.6238 - val_loss: 0.5746 - val_accuracy: 0.6976

KeyboardInterrupt: 

---
Bidirectional LTSM model
---
---

In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(3,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint2])


---
1D Convolutional model
---
---

In [ ]:
from keras import regularizers
model3 = Sequential()
model3.add(layers.Embedding(max_words, 40, input_length=max_len))
model3.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model3.add(layers.MaxPooling1D(5))
model3.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model3.add(layers.GlobalMaxPooling1D())
model3.add(layers.Dense(3,activation='softmax'))
model3.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
checkpoint3 = ModelCheckpoint("best_model3.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model3.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint3])
